# 文本摘要

在上述文本生成的例子中，我们了解到，若要完成一个制定的任务，需要在提示词中加以约束，明确任务类型，本例中要完成文本摘要任务，需要做到“文本”和“提示摘要任务”同时在提示词中。

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [ ]:
text="LLM是大型语言模型（Large Language Model）的缩写，是一个训练大量文本数据的人工智能系统。使用具有数十亿个参数的转换器架构，它学习语言模式以生成连贯的文本、回答问题、翻译等等。它通过统计模式预测下一个符号，但缺乏真正的理解，有时会产生不准确的结果。例子包括GPT和Llama，为聊天机器人和内容工具提供动力。"
prompt=f"""用一个简短的句子描述文本：{text}"""

In [5]:
print(inference(model_instruct,tokenizer_instruct,prompt))

{'response': 'LLM代表大型语言模型，是一种人工智能系统，基于数十亿参数的转换器架构学习语言模式，能够生成文本、回答问题和翻译等，虽然具备生成连贯文本的能力，但缺乏真正理解力，常见于如GPT和Llama等聊天机器人和内容工具中。', 'generated_tokens_count': 64}
